## **1. Package Installations**

In [1]:
!pip install transformers

In [3]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [6]:
!pip install langchain-community langchain-core

In [60]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=e1ea79590a3bf9d43506529023c1b52fda858d8c2eacd64cf804eda6341642ac
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


## **2. Imports**

In [7]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

In [8]:
from datasets import load_dataset
dataset = load_dataset("ncbi/ncbi_disease")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.70k [00:00<?, ?B/s]

ncbi_disease.py:   0%|          | 0.00/5.83k [00:00<?, ?B/s]

The repository for ncbi/ncbi_disease contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ncbi/ncbi_disease.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/5433 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/924 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/941 [00:00<?, ? examples/s]

## **3. Dataset**

In [9]:
# Accessing the data
train_data = dataset['train']
dev_data = dataset['validation']
test_data = dataset['test']

### **4. BioClinicalRobertA model**

In [15]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("mrm8488/bioclinical-roberta-es-finenuned-clinical-ner")
model = AutoModelForTokenClassification.from_pretrained("mrm8488/bioclinical-roberta-es-finenuned-clinical-ner")

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/998k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/540k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/502M [00:00<?, ?B/s]

In [35]:
# spanish text for performing NER
text = "El paciente presenta fiebre alta y dolor abdominal."

In [36]:
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

In [37]:
import torch

with torch.no_grad():
    outputs = model(**inputs)

predictions = torch.argmax(outputs.logits, dim=2)

In [38]:
predicted_tokens = []
for token, prediction in zip(inputs["input_ids"][0], predictions[0]):
    token = tokenizer.decode(token)
    label = model.config.id2label[prediction.item()]
    predicted_tokens.append((token, label))

# Remove special tokens and their predictions
predicted_tokens = [token for token in predicted_tokens if token[0] not in [tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token]]

In [39]:
for token, label in predicted_tokens:
    print(f"{token}: {label}")

 El: O
 paciente: O
 presenta: O
 fiebre: O
 alta: O
 y: O
 dolor: O
 abdominal: O
.: O


### **Fine-tuning the model**

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
from datasets import load_dataset
from seqeval.metrics import classification_report, f1_score, accuracy_score

In [70]:
# Load NCBI Disease dataset
dataset = load_dataset("ncbi_disease")

# Reduce dataset size, as 1000 samples is taking longer time...
dataset['train'] = dataset['train'].select(range(30))
dataset['validation'] = dataset['validation'].select(range(10))

# Define id2label and label2id
label_list = dataset['train'].features['ner_tags'].feature.names
id2label = {i: label for i, label in enumerate(label_list)}
label2id = {v: k for k, v in id2label.items()}

# Set maximum sequence length
max_length = 128

# Load model and tokenizer
model_name = "mrm8488/bioclinical-roberta-es-finenuned-clinical-ner"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at mrm8488/bioclinical-roberta-es-finenuned-clinical-ner and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([23, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([23]) in the checkpoint and torch.Size([3]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### **Helper Functions**

In [75]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        max_length=max_length,
        padding="max_length"
    )
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)


def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(-1)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = classification_report(true_labels, true_predictions)
    return {
        "accuracy": accuracy_score(true_labels, true_predictions),
        "f1": f1_score(true_labels, true_predictions),
        "classification_report": results,
    }

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/941 [00:00<?, ? examples/s]

In [71]:
dataset['train'].features['ner_tags'].feature.names
id2label
label2id

{'O': 0, 'B-Disease': 1, 'I-Disease': 2}

### **Training args and trainer**

In [72]:
# Use DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-72-8af5328ca003>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [73]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Classification Report
1,1.099400,1.011678,0.894495,0.000000,precision recall f1-score support Disease 0.00 0.00 0.00 5 micro avg 0.00 0.00 0.00 5 macro avg 0.00 0.00 0.00 5 weighted avg 0.00 0.00 0.00 5
2,0.583200,0.434307,0.944954,0.000000,precision recall f1-score support Disease 0.00 0.00 0.00 5 micro avg 0.00 0.00 0.00 5 macro avg 0.00 0.00 0.00 5 weighted avg 0.00 0.00 0.00 5
3,0.254600,0.259928,0.944954,0.000000,precision recall f1-score support Disease 0.00 0.00 0.00 5 micro avg 0.00 0.00 0.00 5 macro avg 0.00 0.00 0.00 5 weighted avg 0.00 0.00 0.00 5
4,0.287600,0.243042,0.944954,0.000000,precision recall f1-score support Disease 0.00 0.00 0.00 5 micro avg 0.00 0.00 0.00 5 macro avg 0.00 0.00 0.00 5 weighted avg 0.00 0.00 0.00 5
5,0.075100,0.155592,0.944954,0.000000,precision recall f1-score support Disease 0.00 0.00 0.00 5 micro avg 0.00 0.00 0.00 5 macro avg 0.00 0.00 0.00 5 weighted avg 0.00 0.00 0.00 5


Trainer is attempting to log a value of "              precision    recall  f1-score   support

     Disease       0.00      0.00      0.00         5

   micro avg       0.00      0.00      0.00         5
   macro avg       0.00      0.00      0.00         5
weighted avg       0.00      0.00      0.00         5
" of type <class 'str'> for key "eval/classification_report" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "              precision    recall  f1-score   support

     Disease       0.00      0.00      0.00         5

   micro avg       0.00      0.00      0.00         5
   macro avg       0.00

TrainOutput(global_step=40, training_loss=0.6018340082839131, metrics={'train_runtime': 356.1554, 'train_samples_per_second': 0.421, 'train_steps_per_second': 0.112, 'total_flos': 9798716966400.0, 'train_loss': 0.6018340082839131, 'epoch': 5.0})

### **Evaluation**

In [74]:
test_results = trainer.evaluate(tokenized_datasets["test"])
print(f"Test results: {test_results}")

test_predictions = trainer.predict(tokenized_datasets["test"])
pred_labels = test_predictions.predictions.argmax(-1)

for i in range(5):
    tokens = tokenized_datasets["test"][i]["tokens"]
    true_labels = [id2label[label] for label in tokenized_datasets["test"][i]["ner_tags"]]
    pred_labels_sample = [id2label[label] for label in pred_labels[i][:len(tokens)]]
    print(f"\nSample {i+1}:")
    print("Tokens:", tokens)
    print("True labels:", true_labels)
    print("Predicted labels:", pred_labels_sample)

Trainer is attempting to log a value of "              precision    recall  f1-score   support

     Disease       0.32      0.02      0.03       960

   micro avg       0.32      0.02      0.03       960
   macro avg       0.32      0.02      0.03       960
weighted avg       0.32      0.02      0.03       960
" of type <class 'str'> for key "eval/classification_report" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Test results: {'eval_loss': nan, 'eval_accuracy': 0.9193054136874361, 'eval_f1': 0.031683168316831684, 'eval_classification_report': '              precision    recall  f1-score   support\n\n     Disease       0.32      0.02      0.03       960\n\n   micro avg       0.32      0.02      0.03       960\n   macro avg       0.32      0.02      0.03       960\nweighted avg       0.32      0.02      0.03       960\n', 'eval_runtime': 368.934, 'eval_samples_per_second': 2.551, 'eval_steps_per_second': 0.64, 'epoch': 5.0}

Sample 1:
Tokens: ['Clustering', 'of', 'missense', 'mutations', 'in', 'the', 'ataxia', '-', 'telangiectasia', 'gene', 'in', 'a', 'sporadic', 'T', '-', 'cell', 'leukaemia', '.']
True labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B-Disease', 'I-Disease', 'I-Disease', 'O', 'O', 'O', 'B-Disease', 'I-Disease', 'I-Disease', 'I-Disease', 'I-Disease', 'O']
Predicted labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

Sample 2:
Tokens: ['Atax